In [20]:
import sqlite3
import pandas as pd

# -------------------------------
# Utility: Iterative filter function to drop columns/rows with high missingness
# -------------------------------
def iterative_filter(df, col_thresh=1, row_thresh=1, max_iter=10):
    iteration = 0
    while iteration < max_iter:
        iteration += 1
        col_missing = df.isnull().mean()
        row_missing = df.isnull().mean(axis=1)
        
        cols_to_drop = col_missing[col_missing > col_thresh].index.tolist()
        rows_to_drop = row_missing[row_missing > row_thresh].index.tolist()
        
        if not cols_to_drop and not rows_to_drop:
            break
        
        if cols_to_drop:
            print(f"Iteration {iteration}: Dropping {len(cols_to_drop)} columns with missing fraction > {col_thresh}")
            df = df.drop(columns=cols_to_drop)
        elif rows_to_drop:
            print(f"Iteration {iteration}: Dropping {len(rows_to_drop)} rows with missing fraction > {row_thresh}")
            df = df.drop(index=rows_to_drop)
    return df

# -------------------------------
# Function: Load country metadata with optional filtering by Income Group and/or Region
# -------------------------------
def load_country_meta(db_path, selected_income_group=None, selected_region=None):
    conn = sqlite3.connect(db_path)
    query = "SELECT [Short Name], [Income Group], Region FROM Country"
    filters = []
    if selected_income_group:
        filters.append(f"[Income Group] = '{selected_income_group}'")
    if selected_region:
        filters.append(f"Region = '{selected_region}'")
    if filters:
        query += " WHERE " + " AND ".join(filters)
    df_country_meta = pd.read_sql_query(query, conn)
    conn.close()
    return df_country_meta

# -------------------------------
# Function: Load and filter CSV data based on available countries in meta data
# -------------------------------
def load_and_filter_csv(csv_path, country_list):
    df = pd.read_csv(csv_path)
    print("Initial CSV shape:", df.shape)
    # Keep only rows for countries in the provided country_list
    df = df[df['CountryShortName'].isin(country_list)]
    # Apply iterative filter on the data
    df_filtered = iterative_filter(df.copy())
    print("Filtered CSV shape:", df_filtered.shape)
    return df_filtered

# -------------------------------
# Function: Build indicator mapping from the database
# -------------------------------
def build_indicator_mapping(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE '%Indicator%'")
    tables = [row[0] for row in cursor.fetchall()]
    
    indicator_mapping = {}
    for table in tables:
        if table.startswith("QPSD"):
            col_name = "Series Name"
        else:
            col_name = "Indicator Name"
        query = f"SELECT [{col_name}] FROM [{table}]"
        try:
            cursor.execute(query)
            rows = cursor.fetchall()
            for row in rows:
                indicator_name = row[0]
                if indicator_name:
                    indicator_mapping[indicator_name] = table
        except Exception as e:
            print(f"Error querying table {table} (column '{col_name}'): {e}")
    conn.close()
    return indicator_mapping

# -------------------------------
# Function: Build the indicator mapping report DataFrame
# -------------------------------
def build_indicator_report(df, indicator_mapping):
    # Exclude 'CountryShortName' and 'year'
    indicator_cols = [col for col in df.columns if col not in ['CountryShortName', 'year']]
    report_data = []
    n_rows = len(df)
    
    for indicator in indicator_cols:
        source_table = indicator_mapping.get(indicator, "Unknown Source")
        missing_count = df[indicator].isnull().sum()
        missing_percentage = (missing_count / n_rows) * 100
        report_data.append({
            "Indicator": indicator,
            "Source Table": source_table,
            "Missing Count": missing_count,
            "Missing Percentage": missing_percentage
        })
    
    report_df = pd.DataFrame(report_data)
    report_df = report_df.sort_values(by=["Source Table", "Indicator"]).reset_index(drop=True)
    return report_df


# -------------------------------
# Function: Compute missing values aggregated by country
# -------------------------------
def compute_missing_by_country(df):
    def missing_info(group):
        missing_count = group.isnull().sum().sum()
        total_cells = group.shape[0] * group.shape[1]
        missing_percentage = (missing_count / total_cells) * 100
        return pd.Series({"missing_count": missing_count, "missing_percentage": missing_percentage})
    
    missing_total_by_country = df.groupby("CountryShortName").apply(missing_info).reset_index()
    missing_total_by_country = missing_total_by_country.sort_values("missing_count", ascending=False)
    return missing_total_by_country

# -------------------------------
# Function: Merge missing count with country metadata
# -------------------------------
def merge_with_country_meta(missing_df, df_country_meta):
    merged_report = missing_df.merge(df_country_meta, left_on="CountryShortName", right_on="Short Name", how="left")
    merged_report = merged_report.drop(columns=["Short Name"])
    merged_report = merged_report.sort_values("missing_count", ascending=False)
    return merged_report

# -------------------------------
# Main processing function
# -------------------------------
def main(csv_path, db_path, selected_income_group=None, selected_region=None):
    # Load country metadata with optional filters
    df_country_meta = load_country_meta(db_path, selected_income_group, selected_region)
    print("Country metadata shape (after filtering):", df_country_meta.shape)
    
    # Get list of countries from metadata
    selected_countries = df_country_meta['Short Name'].tolist()
    
    # Load CSV and filter by selected countries
    df_filtered = load_and_filter_csv(csv_path, selected_countries)
    
    # Build indicator mapping report
    indicator_mapping = build_indicator_mapping(db_path)
    report_df = build_indicator_report(df_filtered, indicator_mapping)
    print("\nIndicator Mapping Report:")
    print(report_df)
    
    # Compute missing values aggregated by country
    missing_total_by_country = compute_missing_by_country(df_filtered)
    print("\nMissing Values by Country:")
    print(missing_total_by_country)
    
    # Merge with country metadata
    merged_report = merge_with_country_meta(missing_total_by_country, df_country_meta)
    print("\nFinal Merged Report (Missing Values with Income Group and Region):")
    print(merged_report)
    
    return df_filtered, report_df, merged_report

# -------------------------------
# Run the main processing function with parameters
# -------------------------------
if __name__ == "__main__":
    CSV_PATH = r'D:\GitHub Repos\culminating-project-group-1\Week 3\JC\Group1Data.csv'
    DB_PATH = r"D:\GitHub Repos\culminating-project-group-1\Week 2\Database Files\BANA698GROUP1.db.db"
    
    # Example: Select only countries in a given income group or region.
    # You can set one or both to a specific value or None to skip filtering.
    SELECTED_INCOME_GROUP = None   # or None if you want all
    SELECTED_REGION = None # or None if you want all
    
    df_filtered, report_df, merged_report = main(
        CSV_PATH, DB_PATH,
        selected_income_group=SELECTED_INCOME_GROUP,
        selected_region=SELECTED_REGION
    )


Country metadata shape (after filtering): (265, 3)
Initial CSV shape: (795, 1513)
Filtered CSV shape: (795, 1513)

Indicator Mapping Report:
                                              Indicator  \
0     Gross PSD, Budgetary Central Gov., All maturit...   
1     Gross PSD, Budgetary Central Gov., All maturit...   
2     Gross PSD, Budgetary Central Gov., All maturit...   
3     Gross PSD, Budgetary Central Gov., All maturit...   
4     Gross PSD, Budgetary Central Gov., All maturit...   
...                                                 ...   
1507  Vulnerable employment, male (% of male employm...   
1508  Vulnerable employment, total (% of total emplo...   
1509  Wage and salaried workers, female (% of female...   
1510  Wage and salaried workers, male (% of male emp...   
1511  Wage and salaried workers, total (% of total e...   

                          Source Table  Missing Count  Missing Percentage  
0     QPSDBudgetaryCentralGovIndicator            698           87.798742 